In [39]:
import os

In [40]:
%pwd

'c:\\Users\\Ivar\\Downloads'

In [41]:
os.chdir('../')


In [42]:
%pwd

'c:\\Users\\Ivar'

In [43]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path : Path
    model_path : Path
    tokenizer_path : Path
    metric_file_name : Path 


In [44]:
from textsummarzer.constants import CONFIG_FILE_PATH,CONFIG_FILE_PATH,PARAMS_FILE_PATH
from textsummarzer.utils.common import read_yaml,create_directory

In [45]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        
        create_directory([self.config.artifacts_root]) 
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        create_directory([config.root_dir])
        model_evaluation_config= ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_name=config.metric_file_name,
        )
        return model_evaluation_config

In [46]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset,load_from_disk
import torch 
import tqdm
import pandas as pd


In [47]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    # Evaluation
    
    
    def generate_batch_sized_chunks(list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]



    def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                                batch_size=16, device="cpu",
                                column_text="article",
                                column_summary="highlights"):
        article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):

            inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                            padding="max_length", return_tensors="pt")

            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device),
                            length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

            # Finally, we decode the generated texts,
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                    clean_up_tokenization_spaces=True)
                for s in summaries]

            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


            metric.add_batch(predictions=decoded_summaries, references=target_batch)

        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        return score
  
    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
       
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)


        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
  
        rouge_metric = load_metric('rouge')

        score = self.calculate_metric_on_test_ds(
        dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
            )

        rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

        df = pd.DataFrame(rouge_dict, index = ['pegasus'] )
        df.to_csv(self.config.metric_file_name, index=False)  

In [48]:
try:
    config=ConfigurationManager()
    DataEvalConfig=config.get_model_evaluation_config()
    DataTrainConfig=ModelEvaluation(config=DataEvalConfig)
    DataTrainConfig.evaluate()

except Exception as e:
    raise e

[2024-10-20 17:42:12,958:INFO:common:yaml file: C:\Users\Ivar\Downloads\text summarzre\text-summarizer\config\config.yaml loaded successfully]
[2024-10-20 17:42:12,963:INFO:common:yaml file: C:\Users\Ivar\Downloads\text summarzre\text-summarizer\params.yaml loaded successfully]
[2024-10-20 17:42:12,966:INFO:common:directory: artifacts created]
[2024-10-20 17:42:12,969:INFO:common:directory: artifacts/model_evaluation created]


OSError: Incorrect path_or_model_id: 'artifacts/model_trainer/samsum_dataset'. Please provide either the path to a local folder or the repo_id of a model on the Hub.